In [ ]:
import numpy as np
import pandas as pd
import random
import copy

the algorithm parameters which need to be set prior to the algorithm are :    
B = number of bees    
NC = number of constructive moves during one forward pass     

In [ ]:
B = 10
NC = 25

## helper functions for (partial) solution improvement variations

In [ ]:
def swap(x, idx1, idx2):
    l = x.copy()
    l[idx1], l[idx2] = l[idx2], l[idx1]
    return l

#Creates transposition neighbourhood for x
def transposes(x):
    #neighbours of same number of selected objects
    a = [ swap(x, i, j) for i in range(len(x)) for j in range(i+1, len(x)) ]
    return  a
def swaps(x):
    #neighbours of same number of selected objects
    a = [ swap(x, i, (i+1)%len(x)) for i in range(len(x)) ] 
    return  a

# Hill_Climbing transposition neighbourhood
def hc_t(a, distances):
    threshold = 80  # threshold for neigborhood mode  

    while True:
        if len(a) < threshold:
            neighbours = transposes(a) # create full transposition neighbourhood
        else:
            neighbours = swaps(a)
        if not neighbours: break
        costs = objective_function(distances, neighbours) 
        
        if min(costs) >= objective_function(distances, [a]): break
        a = neighbours[costs.index(min(costs))]
    return a  # solution, optimal performance/gain we get

In [ ]:
# all bees start at a hive

### Initialization --> every bee is set to an empty solution

## forward pass    
1. set k = 1 // counter for constructive moves in the forward pass    
2. evaluate all possible construcive moves    
3. according to evaluation, chose one move using roulette wheel    
4. k = k+1; If k<= NC go to step 2.     

In [ ]:
def forward_pass(NC, bee, costs):
    k = 0
    while k < NC:
        if not bee:
                starting_node = random.choice(range(costs.shape[0]))
                bee.append(starting_node)
                
        if k == 0:
            not_visited = [ind for ind in range(costs.shape[0]) if not(ind in bee)]
        
        next_node = roulette_wheel(costs[bee[-1],[not_visited]]) #[-1] is the last element of the list
        current_node = not_visited[next_node]
        bee.append(current_node)
        k = k+1
        del not_visited[next_node]
    return not_visited, bee

In [ ]:
def roulette_wheel(costs):
    costs_new = 1 / np.array(costs) # to turn around scaling
    normalized_costs = costs_new / np.sum(costs_new)
    bins = np.cumsum(normalized_costs)# returns an array of same length. At each spot is gives the sum of all alements before added up
    roulette_value = [np.random.uniform(0,1)] # draw random value between 0 and 1 
    index_next_node = np.digitize(roulette_value, bins)[0]  # states in which bin the value belongs
    return(index_next_node)

# all bees are back to hive

## backward pass    
sort the bees by their objective function values    
every bee decides randomly if she continues its own exploration or follows another one (higher value higher prob for staying)    
for every follower: choose a new solution from recruiter by roulette wheel    
if stopping condition is not met go to forward pass    
else output best result    

In [ ]:
def backward_pass(costs, bees, count):
    objective_value = objective_function(costs, bees)
    best_bees = []
    sort_bee_indices = np.argsort(objective_value)
    best_bees_index = np.array(sort_bee_indices[0:round(B*0.7)])  # keep the best 70% solutions 

    best_bees.append([bees[i] for i in best_bees_index])

    flat_best_bees = [item for sublist in best_bees for item in sublist]
    objective_values = objective_function(costs, flat_best_bees)

    highest_bee = np.max(objective_value)
    lowest_bee = np.min(objective_value)
    
    O_bee = []
    #normalize: C_max - C_bee / C_max - C_min
    for i, bee in enumerate(flat_best_bees):
        O_bee.append((highest_bee - objective_value[i]) / (highest_bee - lowest_bee))

    O_max = np.max(O_bee)
     
    # Loyaliy
    # calculate random number between 0 and 1. If the number is < probability it stays loyal else uncommitted
    loyalty_bee = [[] for i in range(costs.shape[0])]
    
    # bees with very good solutions stay on their path, no matter what
    for i in np.array(sort_bee_indices[0:round(B*0.1)]):
        loyalty_bee[i] = 0
    
    for i, bee in enumerate(flat_best_bees[round(B*0.1)+1:round(B*0.7)]):
        # probability to be loyal for average bees
        probability = np.exp(-(O_max - O_bee[i])/ count)
        loyalty_value = np.random.uniform(0,1)
        if probability >= loyalty_value:
            loyalty_bee[i] = 1  # stays loyal
        else:
            loyalty_bee[i] = 0 # becomes uncomitted
    # bees with bad solutions become followers no matter what
    for i in np.array(sort_bee_indices[round(B*0.7)+1:B-1]):
        loyalty_bee[i] = 0
        
            
    # recruitment
    # for all bees with value zero: perform roulette wheel (given all best bees) to find recrouter
    # recrouters_index = [i for i, e in enumerate(loyalty_bee) if e != 0]
    recrouters = [objective_value[i] for i in best_bees_index]
    for index, item in enumerate(loyalty_bee):
        if item == 0:
            recrouter = roulette_wheel(recrouters) # only the bees with the 70% best (partial) solutions are put in the roulette wheel
            bees[index] = copy.copy(bees[best_bees_index[recrouter]]) # give the bee the same partial solution its recrouter has
    return bees

In [ ]:
def objective_function(costs, bees):
    value_sum = []
    for bee in bees:
        values = []
        for i in range(len(bee)-1):
            value = costs[bee[i]][bee[i+1]] # get costs from current point to next point
            values.append(value)
        value_sum.append(sum(values))
    return value_sum

In [ ]:
def chunks(l, n):
    # creates chunks of a specific size
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [ ]:
#bees = [[0] for i in range(B)] #[0] cause they start at the hive
bees = [[] for i in range(B)]
# for bees which do not start at the hive:



#example tsp
with open('2.tsp','r') as text:
    tsp_read = text.read()
    numbers = [int(i) for i in tsp_read.split()]
# splits the list into a 150 x 150 list
costs = np.array(list(chunks(numbers, 150)))

# do that for all bees
last_loop = 0
stop_cond = False
count = 0
p_imp_iter = 5
while not stop_cond:
    bs = []
    for bee in bees:
        if last_loop:
            stop_cond = True
            left_nodes, b = forward_pass(last_loop, bee, costs)
        else:
            left_nodes, b = forward_pass(NC, bee, costs)
        bs.append(b)
    bees = bs
    # partial solution improvement
    for i in range(p_imp_iter):
        bees = [hc_t(bee, costs) for bee in bees]
        
    if len(left_nodes) < NC:
        last_loop = len(left_nodes)
    if len(left_nodes) == 0:
        break
    count = count+1
    if not last_loop:
        bees = backward_pass(costs, bees, count)

    
final_objectives = objective_function(costs, bees)
minimal_bee_value = np.min(final_objectives)
minimal_solution = bees[np.argmin(final_objectives)]

# print(minimal_solution, minimal_bee_value)
# print(len(minimal_solution))